# Оконные функции
## Юнит 5. РАБОТА С БАЗАМИ ДАННЫХ. SQL
### Skillfactory: DSPR-19

### 6.1. Что такое оконные функции?

Оконная функция выполняет вычисления для набора строк, некоторым образом связанных с текущей строкой. Можно сравнить её с агрегатной функцией, но у них есть различия: при использовании оконной функции несколько строк не группируются в одну, а продолжают существовать раздельно. А внутри оконная функция, как и агрегатная, может обращаться не только к текущей строке результата запроса.

Пример  
Давайте на простом примере посмотрим, как это работает. Следующий запрос вам уже знаком: он считает общую массу грузов доставленных по городам. Для вычисления используется агрегатная функция sum.

In [ ]:
select 
        c.city_name,
        sum(s.weight) city_weight
from 
        shipping.shipment s
        join shipping.city c on s.city_id = c.city_id
group by 1
order by 2 desc

Предположим, нам захотелось посчитать, каков вклад каждой доставки в общую массу доставок в город, то есть отношение массы конкретной доставки к массе всех доставок в городе.

Решим эту задачу с использованием уже знакомых нам функций.

In [ ]:
with city_weight as 
(
        select 
            c.city_id,
            sum(s.weight) city_weight
        from shipping.shipment s
        join shipping.city c on s.city_id = c.city_id
        group by 1
)
select s.ship_id,s.weight/w.city_weight ratio
from shipping.shipment  s
join city_weight  w on s.city_id = w.city_id
order by 2 desc

Что мы сделали? Сначала агрегатной функцией в подзапросе мы посчитали общую массу доставок в городе, затем посчитали отношение для каждой строки.

Но задачу можно решить в один запрос, используя оконные функции.

In [ ]:
select 
        s.ship_id,
        s.weight/sum(s.weight) over(partition by s.city_id) ratio
from shipping.shipment  s
order by 2 desc

Чтобы посчитать массу всех доставок в городе, мы использовали выражение

In [ ]:
sum(s.weight) over(partition by s.city_id)

Синтаксис
Разберемся с  синтаксисом. Сначала идет оконная функция sum, далее в скобках — значение, к которому она применяется, далее в выражении over указывается то, к каким строкам и по каким правилам она будет применяться. В нашем случае оконная функция применяется ко всем строкам, в разбивке по s.city_id, т. е. по каждому городу. Разделив массу каждой доставки на сумму масс в городе, мы и получим нужную нам величину.

В общем виде синтаксис можно описать следующим образом.

имя_функции (выражение) [ FILTER ( WHERE условие_фильтра ) ] OVER ( определение_окна ).

FILTER ( WHERE условие_фильтра ) работает так же, как и where в обычном запросе, но фильтрация происходит только для вычисления оконной функции, т. е. количество выведенных строк не изменится, а поменяется лишь значение вычисляемой функции.

Вызов оконной функции всегда содержит предложение OVER, следующее за названием и аргументами оконной функции. Это синтаксически отличает её от обычной или агрегатной функции. Предложение OVER определяет, как именно нужно разделить строки запроса для обработки оконной функцией.

Предложение PARTITION BY, дополняющее OVER, указывает, что строки нужно разделить по группам или разделам, объединяя одинаковые значения выражений PARTITION BY. Оконная функция вычисляется по строкам, попадающим в один раздел с текущей строкой.  

### 6.2. Функции row_number, rank, dense_rank
Функции row_number, rank, dense_rank созданы для нумерации и ранжирования данных в таблицах.

Давайте решим задачу нумерации названий городов. Предположим, у нас нет идентификатора city_id и мы хотим его завести. Как мы уже знаем, в справочнике есть одноименные города, и функции row_number, rank и dense_rank ведут себя по-разному.

Выполните запрос в Metabase и посмотрите на результат.



In [ ]:
select 
        c.city_name,
        c.state,
        row_number()over (),
        rank()over (),
        dense_rank() over()
from shipping.city c
order by 1

В данном запросе не указано предложение OVER, то есть мы не разбиваем строки на группы и никак их не сортируем. В таком случае row_number вернет нам какие-то случайные неповторяющиеся значения, причем от выполнения к выполнению у нас могут отличаться значения функции для конкретного города. rank и dense_rank в таком случае вернут 1, т. к. в запросе не указано, по какому полю ранжировать.

Давайте добавим поле, по которому ранжировать. Для этого в предложении OVER нам нужно указать ORDER BY <набор значений, по которому будем фильтровать>.



In [ ]:
select 
        c.city_name,
        c.state,
        row_number()over (order by c.city_name),
        rank()over (order by c.city_name),
        dense_rank() over(order by c.city_name)
from shipping.city c
order by 1

Выполнив такой запрос в Metabase, мы увидим, что row_number начинается с 1 и соответствует номеру строки в запросе, то есть возрастает с шагом 1 в сортировке по имени. Но и тут мы можем получить разные значения для разных вызовов одного и того же запроса, поскольку у нас нет сортировки по штату.

Давайте ее добавим, указав через запятую в конструкции OVER (order by) название штата. Также можно добавлять ключевые слова asc/desc для указания порядка сортировки.

In [ ]:
select 
        c.city_name,
        c.state,
        row_number()over (order by c.city_name, c.state)
from shipping.city c
order by 1

Значение функции dense_rank тоже возрастает с шагом 1, но для одинаковых значений имени города он совпадает и результат будет одинаковым при каждом вызове. Rank также возрастает с шагом 1, если значения уникальны, но в случае повторяющихся значений шаг увеличивается на количество совпадающих значений. Для обоих городов Albany мы видим одинаковое значение "4", но для следующего Albuquerque уже значение "6", так как Albany встретилось 2 раза.  Если повторений больше, то и шаг становится больше. Промотайте результат выполнения до городов Lakewood и увидите прирост в значении на 4, потому что в справочнике целых четыре города с названием Lakewood.

Теперь попробуем разбить нашу алфавитную нумерацию по группам — для каждого штата по отдельности. Для этого в предложение OVER необходимо добавить partition by <набор полей>. По механике это похоже на работу с group by, то есть выделенный набор полей определяет группу строк с одинаковыми значениями этих полей, и все вычисления выполняются в каждой группе отдельно.

In [ ]:
select 
        c.city_name,
        c.state,
        row_number() over (partition by c.state order by c.city_name),
        rank() over (partition by c.state order by c.city_name),
        dense_rank() over (partition by c.state order by c.city_name)
from shipping.city c
order by c.state,c.city_name

Так как у нас внутри одного штата нет одноименных городов, то и значения rank,dense_rank и row_number совпадают. 

Оконные функции можно писать только в разделах SELECT и ORDER BY, при этом нельзя вызвать оконную функцию от оконной функции. При необходимости более сложной логики используйте подзапросы. Оконные функции сочетаются с известными нам подзапросами, математическими, строковыми и другими функциями, кроме агрегатных.

### Задание 6.2.1
Проранжируйте уникальными числами имена клиентов в порядке обратном алфавитному, а затем отсортируйте запрос по столбцу с номером по возрастанию. Столбцы в выдаче:

cust_id — id клиента  
cust_name — имя клиента  
num — порядковый номер  

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select
    c.cust_id,
    c.cust_name,
    row_number() over( order by c.cust_name desc) num
from
    shipping.customer c

### Задание 6.2.2
Предположим, вы хотите устроить акцию и вернуть бонусами деньги за доставку трех самых тяжелых грузов для каждого клиента. Напишите запрос, который отранжирует все заказы для каждого клиента по массе груза по убыванию, и выберите 3 самых тяжелых из них. Сортировка по cust_id, а затем по столбцу с номером по возрастанию. Столбцы в выдаче:

cust_id  
ship_id  
weight  
weight_number (row_number для заказа по массе по клиенту)  

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select
    *
from
    (select
        s.cust_id,
        s.ship_id,
        s.weight,
        row_number() over ( partition by s.cust_id  order by s.weight desc ) r
    from
        shipping.shipment s
    ) x
where
    x.r <=3

### 6.3. Конструкция window
Давайте вернемся к нашему запросу:



In [ ]:
select 
        c.city_name,
        c.state,
        row_number() over ( partition by c.state order by c.city_name),
        rank() over ( partition by c.state order by c.city_name),
        dense_rank() over ( partition by c.state order by c.city_name)
from shipping.city c
order by c.state,c.city_name


Как мы видим, предложение OVER одинаковое для всех функций, но мы можем менять их, и они не будут влиять на результаты других функций. 

In [ ]:
select 
    c.city_name,
        c.state,
        row_number() over ( order by c.city_name),
        rank() over ( partition by c.state ),
        dense_rank() over ( partition by c.state order by c.city_name desc)
from shipping.city c
order by c.state,c.city_name

Запрос выше содержит совсем разное определение «окон»: это очень удобно, если вы хотите посчитать несколько значений по различной логике. Например, следующий запрос выводит первый и последний по алфавиту город:

In [ ]:
with rownums as 
(
select 
        c.city_name,
        row_number()over ( order by c.city_name) ascending,
        row_number()over ( order by c.city_name desc) descending
from shipping.city c
order by c.city_name
)
select 
        r.city_name
from 
        rownums r 
where 
        r.ascending = 1 or r.descending = 1

Если у нас повторяется определение окна, при внесении изменений совсем не хочется менять каждую строчку запроса. Для этого существует конструкция, позволяющая задать описание окна один раз и использовать его в нескольких оконных функциях:

In [ ]:
select 
        c.city_name,
        c.state,
        row_number() over my_window,
        rank()over my_window,
        dense_rank() over my_window
from shipping.city c
window my_window as ( partition by c.state order by c.city_name)
order by c.state,c.city_name

Конструкция window <имя окна> as <определение окна> серьезно упрощает работу с большим числом оконных функций. Окна, как и подзапросы в конструкции перечисляются через запятую, например:

In [ ]:
select 
        c.city_name,
        c.state,
        row_number() over my_window,
        rank()over my_window,
        dense_rank() over my_window1
from shipping.city c
window my_window as ( partition by c.state order by c.city_name),
 my_window1 as ( partition by c.state order by c.city_name desc)
order by c.state,c.city_name

### Задание 6.3.1
Давайте попрактикуемся в использовании общего окна для нескольких функций. Выведите результат ранжирования клиентов по годовой выручке по убыванию функциями row_number, rank, dense_rank, используя общее окно. Столбцы в выдаче:

cust_name  
row_number  
rank  
dense_rank  
annual_revenue  

Сортировка по annual_revenue по убыванию.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select
    c.cust_name,
    row_number() over wind,
    rank() over wind,
    dense_rank() over wind,
    c.annual_revenue
from
    shipping.customer c
window wind as (order by c.annual_revenue desc)

### 6.4. Агрегаты в оконных функциях
Функции min, max, avg, sum, count, которые мы изучали в предыдущих разделах, могут быть реализованы в оконной интерпретации: поле группировки нужно указать не в group by, а в partition by. 

Главное отличие — они будут считаться для каждой строки, а поле, по которму идет группировка, не будет уникальным, если не использовать distinct.

Вот так выглядит запрос с агрегатной функцией:

In [ ]:
select 
    date_trunc('month', s.ship_date) mn,
    count(*) qty
from shipping.shipment s
group by 1
order by 1

А вот ровно тот же запрос, написанный с использованием оконной функции count. Попробуйте выполнить его в Metabase, уберите distinct и убедитесь, что данные не «схлопнуты».

In [ ]:
select distinct 
    date_trunc('month', s.ship_date) mn,
    count(*) over (partition by date_trunc('month', s.ship_date))qty
from shipping.shipment s
order by 1

Сама по себе функция distinct достаточно тяжеловесная, поэтому используйте оконные функции вместо агрегатных только в том случае, если у вас помимо агрегата используются другие оконные функции. 

Например, вам нужно взять первый и последний месяц и сравнить их. Можно написать два подзапроса и использовать UNION, а можно посчитать два dense_rank по убыванию и возрастанию и посчитать количество доставок:

In [ ]:
select 
t.mn,
t.qty
from 
(
select distinct 
date_trunc('month', s.ship_date) mn,
count(s.ship_id) over (partition by date_trunc('month', s.ship_date))qty,
    dense_rank() over (order by date_trunc('month', s.ship_date)) ascend,
        dense_rank() over (order by date_trunc('month', s.ship_date) desc) descend
    from shipping.shipment s
    order by 1
    ) t 
where t.ascend = 1 or t.descend = 1

### Задание 6.4.1
Представим, что мы хотим оценить и сравнить самых «богатых» и самых «бедных» клиентов по выручке. Напишите запрос, который выведет трех лидеров и трех аутсайдеров по выручке, их количество доставок и средний вес доставки. Столбцы в выдаче:

cust_name — имя клиента  
annual_revenue — годовая выручка  
ship_qty — количество доставок на клиента  
avg_weight — средний вес доставки  

Сортировка по annual_revenue по убыванию.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
# не приняло платформой

with a as (select
    c.cust_id,
    c.cust_name,
    c.annual_revenue,
    row_number()over(order by c.annual_revenue desc) desceding
from
    shipping.customer c
order by annual_revenue),

maxx as (
    select
    a.cust_id,
    a.cust_name,
    a.annual_revenue,
    count(s.cust_id),
    avg(s.weight)
from
    a left join shipping.shipment s on a.cust_id = s.cust_id
group by 1,2,3,a.asceding
order by a.asceding 
limit 3),

minn as (
select
    a.cust_id,
    a.cust_name,
    a.annual_revenue,
    count(s.cust_id),
    avg(s.weight)
from
    a left join shipping.shipment s on a.cust_id = s.cust_id
group by 1,2,3,a.desceding
order by a.desceding  
limit 3)

select
    *
from
    maxx
union all
select
    *
from
    minn


**Накопленные значения**  
Помимо удобства сочетания разных функций и сложной логики, оконные агрегатные функции позволяют просто считать накопительные и плавающие метрики, так как в предложении OVER есть возможность сортировки. Давайте посмотрим, как это работает. Выполните запрос в Metabase и посмотрите на результат. Как мы видим, результатом стало кумулятивное число доставок по месяцам. Попробуйте убрать order by из over, и вы получите одинаковое общее число доставок за все месяцы в каждой строке.

In [ ]:
select 
    distinct 
        date_trunc('month',ship_date) mn,
        count(*) over (order by date_trunc('month',ship_date) ) shipments_cumulative
from shipping.shipment
order by 1

### 6.5. Работа с событийными логами

Частый кейс применения оконных функций — работа с событийными логами. Давайте поработаем с такими данными. В схеме webevents есть таблица event_log, в которой лежат события с сайта интернет-магазина со следующими столбцами:

Столбец         Описание  
client_id	    Идентификатор клиента  
event_name	    Тип события (view - просмотр товара, addtocart - добавление товара в корзину, transaction - заказ)  
item_id	        Идентификатор товара  
order_id	    Идентификатор заказа  
event_datetime	Время события   

_Посмотрите ее содержимое в Metabase._

 Давайте выполним простое упражнение по вычислению конверсии из события view в событие addtocart. Для этого вспомним агрегатные функции и поделим число клиентов, которые добавили товар в корзину, на число клиентов, просматривавших любой товар.


In [ ]:
select
    date_trunc('month',el.event_datetime) mn,
    count(distinct case when el.event_name = 'addtocart' then el.client_id end)*1.0/
        count(distinct case when el.event_name = 'view' then el.client_id end) conv
from webevents.event_log el
group by 1

### Задание 6.5.1
Давайте оценим конверсию из добавления в корзину в покупку. Напишите запрос, который выведет коверсию из события addtocart в transaction по дням, а также выведите общее число заказов. Столбцы в выдаче:

dt — дата события  
conv — конверсия из корзины в покупку  
orders_cnt — количество заказов  

Сортировка по дате по возрастанию. Столбец с датой необходимо привести к типу date.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select
date_trunc('day',el.event_datetime)::date dt,
count(distinct case when el.event_name = 'transaction' then el.client_id end)*1.0/
nullif(count(distinct case when el.event_name = 'addtocart' then el.client_id end),0) conv,
count(distinct case when el.event_name = 'transaction' then el.order_id end) orders_cnt 
from webevents.event_log el
group by 1
order by 1

### Задание 6.5.2
Давайте посмотрим в динамике на соотношение событий каждого типа — обычное и накопленное. Напишите запрос, который выведет количество открытий товаров, добавлений в корзину, оформлений заказов в разбивке по месяцам, а также кумулятивно эти метрики. Столбцы в выдаче:

dt — дата первого дня месяца события  
views, carts, orders — 3 столбца, которые содержат количество событий view, addtocart, transaction соответственно в этот месяц  
столбцы views_cumulative, carts_cumulative, orders_cumulative, в которых будут те же значения, но с накоплением  

Сортировка по месяцу по убыванию. Столбец с датой необходимо привести к типу date.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
# не принимает платформой
select
    date_trunc('day',el.event_datetime)::date dt,
    count(case when el.event_name = 'view' then  el.event_name end) over (wind) as view,
    count(case when el.event_name = 'view' then  el.event_name end) over (wind_c) as view_c,
    count(case when el.event_name = 'addtocart' then  el.event_name end) over (wind) as atc,
    count(case when el.event_name = 'addtocart' then  el.event_name end) over (wind_c) as atc_c,
    count(case when el.event_name = 'transaction' then  el.event_name end) over (wind) as tst,
    count(case when el.event_name = 'transaction' then  el.event_name end) over (wind_c) as tst_c
from 
    webevents.event_log el
window wind as ( partition by date_trunc('day',el.event_datetime) ),
wind_c as ( order by date_trunc('day',el.event_datetime) )
order by 1 desc

In [ ]:
# и такой не принимает

select
date_trunc('day',el.event_datetime)::date dt,
count(case when el.event_name = 'view' then  el.event_name end) over (wind) as views,
count(case when el.event_name = 'view' then  el.event_name end) over (wind_c) as views_cumulative,
count(case when el.event_name = 'addtocart' then  el.event_name end) over (wind) as carts,    
count(case when el.event_name = 'addtocart' then  el.event_name end) over (wind_c) as carts_cumulative,
count(case when el.event_name = 'transaction' then  el.event_name end) over (wind) as orders,
count(case when el.event_name = 'transaction' then  el.event_name end) over (wind_c) as orders_cumulative

from 
    webevents.event_log el
window wind as ( partition by date_trunc('day',el.event_datetime) ),
wind_c as ( order by date_trunc('day',el.event_datetime) )
order by 1 desc

In [ ]:
# принимает
select distinct
 date_trunc('month',el.event_datetime)::date dt,
count( case when el.event_name = 'view' then el.client_id end) over pr as views,
    count( case when el.event_name = 'addtocart' then el.client_id end) over pr carts,
    count( case when el.event_name = 'transaction' then el.client_id end) over pr orders,
    count((case when el.event_name = 'view' then el.client_id end)) over mn  views_cumulative,
    count((case when el.event_name = 'addtocart' then el.client_id end)) over mn carts_cumulative,
    count((case when el.event_name = 'transaction' then el.client_id end)) over mn orders_cumulative
from webevents.event_log el
window 
    mn as (order by date_trunc('month',el.event_datetime)),
    pr as (partition by date_trunc('month',el.event_datetime))
order by 1 desc

### 6.6. Функции lead/lag
Функции lead/ lag нужны для получения следующих и предыдущих значений выражения в таблице.

Давайте решим задачу: найдем среднее время между заказами клиента в нашем сервисе доставок.

In [ ]:
with gaps as 
(
select 
    s.cust_id,
    lead(s.ship_date) over (partition by s.cust_id order by s.ship_date) - s.ship_date diff
from 
    shipping.shipment s
)
select 
    c.cust_name,
    avg(g.diff)
from gaps g 
left join shipping.customer c on g.cust_id = c.cust_id
group by 1

Перевести это на человеческий язык можно так:

(partition by s.cust_id order by s.ship_date) — в группировке по пользователю и сортировке по дате
lead(s.ship_date) — берем следующую дату доставки
Далее из следующей вычитается текущая, чтобы найти разницу между двумя доставками, далее полученные значения обычной агрегатной функцией усредняются и получается результат. Для последней доставки пользователя lead(s.ship_date) будет null, так как следующей не существует.
Функция lag работает аналогично, только берет предыдущее значение. Этот же запрос можно переписать следующим образом, используя lag.

In [ ]:
with gaps as 
(
select 
    s.cust_id,
    s.ship_date - lag (s.ship_date) over (partition by s.cust_id order by s.ship_date) diff
from 
    shipping.shipment s
)
select 
    c.cust_name,
    avg(g.diff)
from gaps g 
left join shipping.customer c on g.cust_id = c.cust_id
group by 1

Если есть необходимость делать шаг не в одну, а в другое число записей, можно передать целым числом параметр шага, с которым будет браться следующее/предыдущее значение.

### 6.7. Плавающие окна
Любая из оконных функций может быть ограничена числом строк, на которых будет идти вычисление. По умолчанию ограничение стоит от первой строки сортировки до текущей, поэтому count на каждой новой строке вырастает на значение конкретного месяца. 

Этими параметрами можно управлять, используя такой параметр, как рамки. После описания оконной функции и раздела over можно описать рамки двумя способами: через строки или через интервал.  

**Синтаксис**
Давайте разберем синтаксис.

- имя_функции (выражение) [ FILTER ( WHERE условие_фильтра ) ] OVER ( определение_окна ) { RANGE | ROWS } начало_рамки
- имя_функции (выражение) [ FILTER ( WHERE условие_фильтра ) ] OVER ( определение_окна ) { RANGE | ROWS } { RANGE | ROWS } BETWEEN начало_рамки AND конец_рамки.  

Возможные значения для начала и конца рамки:

- UNBOUNDED PRECEDING
- значение PRECEDING
- CURRENT ROW
- значение FOLLOWING
- UNBOUNDED FOLLOWING  

Определение рамки задаёт набор строк, образующих рамку окна. Рамка окна представляет собой подмножество строк текущего раздела и используется для оконных функций, работающих с рамкой, а не со всем разделом. Рамку можно указать в режимах **RANGE** или **ROWS**; в любом случае она начинается с положения начало_рамки и заканчивается положением конец_рамки. Если конец_рамки опущен, подразумевается CURRENT ROW (текущая строка).

По умолчанию рамка определяется как **RANGE UNBOUNDED PRECEDING**, что равносильно расширенному определению RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW. С указанием ORDER BY это означает, что рамка будет включать все строки от начала раздела до последней строки, родственной текущей (для ORDER BY). Без ORDER BY в рамку включаются все строки раздела, так как все они считаются родственными текущей.

Если начало_рамки задано как **UNBOUNDED PRECEDING**, рамка начинается с первой строки раздела, а если конец_рамки определён как **UNBOUNDED FOLLOWING**, рамка заканчивается последней строкой раздела.

В режиме **RANGE** начало_рамки, заданное как CURRENT ROW, определяет в качестве начала первую родственную строку (строку, которую ORDER BY считает равной текущей), тогда как конец_рамки, заданный как CURRENT ROW, определяет концом рамки последнюю родственную (для ORDER BY) строку.

В режиме **ROWS** вариант CURRENT ROW просто обозначает текущую строку.

Варианты значение **PRECEDING** и значение **FOLLOWING** допускаются только в режиме ROWS. Они указывают, что рамка начинается или заканчивается со сдвигом на заданное число строк перед или после заданной строки. Здесь значение должно быть целочисленным выражением, не содержащим переменные, агрегатные или оконные функции, и может быть нулевым, что будет означать выбор текущей строки.

**Ограничения**  

Действуют также ограничения: начало_рамки не может определяться как UNBOUNDED FOLLOWING, а конец_рамки — UNBOUNDED PRECEDING, и конец_рамки не может определяться раньше, чем начало_рамки — например, запись RANGE BETWEEN CURRENT ROW AND значение PRECEDING недопустима.